## Calculate Phi vertical mean for a particular layer and LME

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib.pyplot as plt

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
a1,a2,b1,b2 = [240,380,125,320]

In [3]:
filename = f'{path}/DPLE-FOSI_hindcast_T_S_O2_1948-2017_monthly.nc'
dset = xr.open_dataset(filename)
dz = dset['dz']/1e2 # m
darea = dset['TAREA']/1e10 #km*km
dvol = dz*darea

In [4]:
KMT = dset['KMT']
nlat = dset.dims['nlat']
nlon = dset.dims['nlon']
for xx in range(nlat):
    for yy in range(nlon):
        kk = np.int(KMT[xx, yy])
        if kk < 60:
            dvol[kk:, xx, yy] = float("nan")
dvol = dvol.isel(z_t=range(0, 35), nlat=range(a1, a2), nlon=range(b1, b2))

In [5]:
#filename = f'{path}/DPLE_driftcorrected_Phi_ens_mean_monthly.nc'
filename = f'{path}/DPLE_driftcorrected_Phi_Eo_space_ens_mean_monthly.nc'
ds = xr.open_dataset(filename)
dsf = xr.Dataset()
for ii in range(13):
    Phi_i = ds[f'Phi_{ii}']*dvol
    # select a vertical layer
    dsf[f'Phi_{ii}_0-200m'] = Phi_i.isel(z_t=range(0, 20, 1)).sum(dim='z_t')/(dvol.isel(z_t=range(0, 20, 1)).sum(dim='z_t'))
    dsf[f'Phi_{ii}_200-600m'] = Phi_i.isel(z_t=range(20, 35, 1)).sum(dim='z_t')/(dvol.isel(z_t=range(20, 35, 1)).sum(dim='z_t'))

In [6]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:          (nlat: 140, nlon: 195, Y: 64, L: 122)
Coordinates:
    ULONG            (nlat, nlon) float64 101.7 102.9 104.0 ... 319.5 320.0
    ULAT             (nlat, nlon) float64 15.13 15.13 15.14 ... 70.63 70.63
  * Y                (Y) int64 1954 1955 1956 1957 1958 ... 2014 2015 2016 2017
  * L                (L) int64 1 2 3 4 5 6 7 8 ... 116 117 118 119 120 121 122
    TLAT             (nlat, nlon) float64 14.96 14.96 14.97 ... 70.42 70.42
    TLONG            (nlat, nlon) float64 101.2 102.3 103.4 ... 319.2 319.7
Dimensions without coordinates: nlat, nlon
Data variables: (12/26)
    Phi_0_0-200m     (Y, L, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_0_200-600m   (Y, L, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_1_0-200m     (Y, L, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_1_200-600m   (Y, L, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_2_0-200m     (Y, L, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_2_200-600m   (Y, L, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    ...               ...
    Phi_10_0-200m    (Y, L, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_10_200-600m  (Y, L, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_11_0-200m    (Y, L, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_11_200-600m  (Y, L, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_12_0-200m    (Y, L, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_12_200-600m  (Y, L, nlat, nlon) float64 nan nan nan nan ... nan nan nan

In [7]:
%time
#dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_vertical_mean_ens_mean_monthly.nc', mode='w')
dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_Eo_space_vertical_mean_ens_mean_monthly.nc', mode='w')

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 5.25 ms


## calculate LME-averaged Phi 

In [3]:
path = f'/glade/scratch/{USER}/DPLE-results'
#filename = f'{path}/DPLE_driftcorrected_Phi_vertical_mean_ens_mean_monthly.nc'
filename = f'{path}/DPLE_driftcorrected_Phi_Eo_space_vertical_mean_ens_mean_monthly.nc'
dsf = xr.open_dataset(filename)

In [4]:
datadir = f'/glade/u/home/{USER}/O2prediction/Prediction/notebooks/data'
filename = '%s/LME-mask-POP_gx1v6.nc' % datadir
ds1 = xr.open_dataset(filename)
lme_mask = ds1['lme_mask']

In [5]:
#lme = range(np.int(lme_mask.min()), np.int(lme_mask.max()), 1)
lme = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 65]
nyear = dsf.dims['Y']
nlead = dsf.dims['year']

In [6]:
boxPhi = xr.Dataset()
#for ii in range(9):
for ii in range(13):
    print(ii)
    phi_box1 = np.ma.zeros([len(lme), nyear, nlead])
    phi_box1.mask = True
    phi_box2 = phi_box1.copy()
    for box, hh in zip(lme, range(len(lme))):
        latloc, lonloc = np.where(lme_mask == box)
        latloc = list(np.array(latloc) - a1)
        lonloc = list(np.array(lonloc) - b1)
        ar1 = np.ma.zeros([len(latloc),nyear,nlead]);ar1.mask=True
        ar2 = ar1.copy()
        for bb in range(len(latloc)):
            ar1[bb] = dsf[f'Phi_{ii}_0-200m'].isel(nlat=latloc[bb], nlon=lonloc[bb])
            ar2[bb] = dsf[f'Phi_{ii}_200-600m'].isel(nlat=latloc[bb], nlon=lonloc[bb])
        phi_box1[hh] = np.nanmean(ar1,axis=0)
        phi_box2[hh] = np.nanmean(ar2,axis=0)
        #phi_box1[hh] = dsf[f'Phi_{ii}_0-200m'].isel(nlat=latloc, nlon=lonloc).mean(dim='nlat').mean(dim='nlon')
        #phi_box2[hh] = dsf[f'Phi_{ii}_200-600m'].isel(nlat=latloc, nlon=lonloc).mean(dim='nlat').mean(dim='nlon')
    boxPhi[f'Phi_{ii}_0-200m'] = xr.DataArray(data=phi_box1, coords=dict(lme=('lme', lme), Y=('Y', dsf.coords['Y'].values), L=('L', dsf.coords['year'].values)), dims=('lme','Y','L'))
    boxPhi[f'Phi_{ii}_200-600m'] = xr.DataArray(data=phi_box2, coords=dict(lme=('lme', lme), Y=('Y', dsf.coords['Y'].values), L=('L', dsf.coords['year'].values)), dims=('lme','Y','L'))

0
1
2
3
4
5
6
7
8
9
10
11
12


In [7]:
boxPhi.load()
dout = f'/glade/scratch/{USER}/DPLE-results'
#boxPhi.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_LME_ens_mean_monthly.nc', mode='w')
boxPhi.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_Eo_space_LME_ens_mean_monthly.nc', mode='w')